In [2]:
#from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly, KNNBasic, NormalPredictor
from surprise import accuracy
from surprise.model_selection import KFold

In [3]:
# 数据读取
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('./ratings.csv', reader=reader)
train_set = data.build_full_trainset()

In [7]:
# ALS优化
bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}
# SGD优化
#bsl_options = {'method': 'sgd','n_epochs': 5}
algo = BaselineOnly(bsl_options=bsl_options)
#algo = BaselineOnly()
#algo = NormalPredictor()

# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

Estimating biases using als...
RMSE: 0.8641
Estimating biases using als...
RMSE: 0.8629
Estimating biases using als...
RMSE: 0.8648
user: 196        item: 302        r_ui = 4.00   est = 3.98   {'was_impossible': False}


In [9]:
from surprise import SVDpp
from surprise import SVD

algos = [SVD(biased=False), SVD(biased=True), SVDpp()]
algo_names = ['funkSVD', 'BiasSVD', 'SVD++']

In [ ]:
# 定义K折交叉验证迭代器，K=3
for i, algo in enumerate(algos):
    kf = KFold(n_splits=3)
    for trainset, testset in kf.split(data):
        # 训练并预测
        algo.fit(trainset)
        predictions = algo.test(testset)
        # 计算RMSE
        print('Using ' + str(algo_names[i]))
        accuracy.rmse(predictions, verbose=True)

    uid = str(196)
    iid = str(302)
    # 输出uid对iid的预测结果
    pred = algo.predict(uid, iid, r_ui=4, verbose=True)

Using funkSVD
RMSE: 0.8741
Using funkSVD
RMSE: 0.8722
Using funkSVD
RMSE: 0.8713
user: 196        item: 302        r_ui = 4.00   est = 4.32   {'was_impossible': False}
Using BiasSVD
RMSE: 0.8445
Using BiasSVD
RMSE: 0.8454
Using BiasSVD
RMSE: 0.8449
user: 196        item: 302        r_ui = 4.00   est = 3.93   {'was_impossible': False}
Using SVD++
RMSE: 0.8326
